# ANN
- 2020-2 Artifitial Intelligence
- week9 assignment
- MNIST layer 변경 
- updated 1101

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive

/content/drive/My Drive


In [4]:
# ready
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
import torch.optim as optim

import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

from keras.datasets import mnist

root_dir = "/AI_assignment"

<pre>
<h2> step1) Load MNIST (X_train, y_train), (X_test, y_test)</h2>
</pre>

In [5]:
def load_dataset():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  print(X_train.shape) #(60000, 28, 28)
  print(y_train.shape)
  print(X_test.shape) #(10000, 28, 28)
  print(y_test.shape)
  X_train = X_train.reshape(-1, 28*28)
  
  print(X_train.shape)
  X_test  = X_test.reshape(-1, 28*28)

  X_train = torch.tensor(X_train, dtype=torch.float)
  y_train = torch.tensor(y_train, dtype=torch.long)
  X_test = torch.tensor(X_test, dtype=torch.float)
  y_test = torch.tensor(y_test, dtype=torch.long)
  return (X_train, y_train), (X_test, y_test)
tmp = load_dataset()

11493376/11490434 [==============================] - 1s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)


<pre>
<h2> step2) NN Model MNIST</h2>
layer_1, layer_3, layer_5, layer_10
</pre>

In [21]:
class MNIST(nn.Module):
  def __init__(self, config):
    super(MNIST, self).__init__()

    # config 
    self.width = config["input_width_size"]
    self.height = config["input_height_size"]
    self.feature_size = config["feature_size"]
    self.num_labels = config["num_labels"]

    self.layer_1 = nn.Sequential(
        nn.Linear(in_features=self.width*self.height, out_features = self.num_labels),
        nn.Sigmoid()
    )

    self.layer_3 = nn.Sequential(
        nn.Linear(in_features=self.width*self.height, out_features = 256),
        nn.Sigmoid(),
        nn.Linear(in_features=256, out_features = 64),
        nn.Sigmoid(),
        nn.Linear(in_features=64, out_features = self.num_labels),
        nn.Sigmoid()
    )

    self.layer_5 = nn.Sequential(
        nn.Linear(in_features=self.width*self.height, out_features = 512),
        nn.Sigmoid(),
        nn.Linear(in_features=512, out_features = 256),
        nn.Sigmoid(),
        nn.Linear(in_features=256, out_features = 128),
        nn.Sigmoid(),
        nn.Linear(in_features=128, out_features = 64),
        nn.Sigmoid(),
        nn.Linear(in_features=64, out_features = self.num_labels),
        nn.Sigmoid()
    )

    self.layer_10 = nn.Sequential(
        nn.Linear(in_features=self.width*self.height, out_features = 600),
        nn.Sigmoid(),
        nn.Linear(in_features=600, out_features = 512),
        nn.Sigmoid(),
        nn.Linear(in_features=512, out_features = 400),
        nn.Sigmoid(),
        nn.Linear(in_features=400, out_features = 256),
        nn.Sigmoid(),
        nn.Linear(in_features=256, out_features = 128),
        nn.Sigmoid(),
        nn.Linear(in_features=128, out_features = 100),
        nn.Sigmoid(),
        nn.Linear(in_features=100, out_features = 64),
        nn.Sigmoid(),
        nn.Linear(in_features=64, out_features = 32),
        nn.Sigmoid(),
        nn.Linear(in_features=32, out_features = 16),
        nn.Sigmoid(),
        nn.Linear(in_features=16, out_features = self.num_labels),
        nn.Sigmoid()
    )

  def forward(self, input_features, labels=None ):
      input_features = self.layer_10(input_features)
      
      if labels is not None:
           loss_fnc = nn.CrossEntropyLoss()
           logits = input_features
           loss = loss_fnc(logits, labels)
           return loss
           
      else:
          output = torch.argmax(input_features, -1)
          return output


In [7]:
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

<pre>
<h2> 2. 불러온 데이터를 이용하여 모델 학습 및 평가 </h2>
</pre>

In [8]:
# test 데이터로 모델을 평가하는 함수
def do_test(model, test_dataloader):
  model.eval()
  predicts, answers = [], []
  for step, batch in enumerate(test_dataloader):
    
    batch = tuple(t.cuda() for t in batch)

    input_features, labels = batch
    output = model(input_features)

    predicts.extend(tensor2list(output))
    answers.extend(tensor2list(labels))
    
  print("Accuracy : {}".format(accuracy_score(answers, predicts)))

In [9]:
def train(config):
  # 모델 생성
  model = MNIST(config).cuda()

  # 데이터 Load
  (X_train, y_train), (X_test, y_test) = load_dataset()
  
  # TensorDataset/DataLoader를 통해 batch 단위로 데이터를 나누고 Shuffle
  train_features = TensorDataset(X_train, y_train)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

  test_features = TensorDataset(X_test, y_test)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])

  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  model.train()
  for epoch in range(config["epoch"]):
    losses = []

    for step, batch in enumerate(train_dataloader):

      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch

      loss = model(input_features, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (step+1) % 1000 == 0:
        print("{} step processed.. current loss : {}".format(step+1, loss.data.item()))
      losses.append(loss.data.item())
    
    print("Average Loss : {}".format(np.mean(losses)))
    # epoch이 끝날 때 마다, 모델 저장
    torch.save(model.state_dict(), os.path.join(config["output_dir_path"], "epoch_{}.pt".format(epoch + 1)))

    # 지금까지 학습한 가중치로 평가 진행
    do_test(model, test_dataloader)
    

def test(config):
  model = MNIST(config).cuda()

  # 저장된 모델 가중치 Load
  model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["trained_model_name"])))

  # 데이터 load
  (_, _), (X_test, y_test) = load_dataset()
  
  test_features = TensorDataset(X_test, y_test)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])
  
  do_test(model, test_dataloader)

### step4) layer_1, layer_3, layer_5, layer_10 을 사용하여 hidden layer 를 늘려가면서 학습을 수행한 수 수행 결과를 비교해본다.

##### layer_1 : hidden layer 1개 사용

In [11]:
import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "trained_model_name":"epoch_{}.pt".format(10),
              "output_dir_path":output_dir,
              "input_width_size":28,
              "input_height_size":28,
              "feature_size": 512,
              "num_labels": 10,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
1000 step processed.. current loss : 1.620339274406433
Average Loss : 1.7344207080841065
Accuracy : 0.8155
1000 step processed.. current loss : 1.6849464178085327
Average Loss : 1.5999470479329427
Accuracy : 0.8497
1000 step processed.. current loss : 1.5320730209350586
Average Loss : 1.5810541371663411
Accuracy : 0.8846
1000 step processed.. current loss : 1.4910818338394165
Average Loss : 1.5723703810373941
Accuracy : 0.8818
1000 step processed.. current loss : 1.5283877849578857
Average Loss : 1.5666840552012125
Accuracy : 0.8936
1000 step processed.. current loss : 1.6521902084350586
Average Loss : 1.5632415227890015
Accuracy : 0.8966
1000 step processed.. current loss : 1.5530924797058105
Average Loss : 1.5608470232009888
Accuracy : 0.8847
1000 step processed.. current loss : 1.60030996799469
Average Loss : 1.5586708885828653
Accuracy : 0.9027
1000 step processed.. current loss : 1.4840083122253418
Average Loss : 1.556

##### layer_3 : hidden layer 3개 사용

In [14]:
import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "trained_model_name":"epoch_{}.pt".format(10),
              "output_dir_path":output_dir,
              "input_width_size":28,
              "input_height_size":28,
              "feature_size": 512,
              "num_labels": 10,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
1000 step processed.. current loss : 1.946298360824585
Average Loss : 1.98272950630188
Accuracy : 0.8848
1000 step processed.. current loss : 1.6984329223632812
Average Loss : 1.7029244096755982
Accuracy : 0.907
1000 step processed.. current loss : 1.6096853017807007
Average Loss : 1.6162887678146363
Accuracy : 0.9242
1000 step processed.. current loss : 1.5693327188491821
Average Loss : 1.5743199388504028
Accuracy : 0.9339
1000 step processed.. current loss : 1.5609551668167114
Average Loss : 1.5485924229303996
Accuracy : 0.9405
1000 step processed.. current loss : 1.4839526414871216
Average Loss : 1.5323645364125569
Accuracy : 0.9447
1000 step processed.. current loss : 1.4872910976409912
Average Loss : 1.5224092310587565
Accuracy : 0.9472
1000 step processed.. current loss : 1.5172085762023926
Average Loss : 1.5151572317123414
Accuracy : 0.9503
1000 step processed.. current loss : 1.558019757270813
Average Loss : 1.50961

##### layer_5 : hidden layer 5개 사용

In [17]:
import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "trained_model_name":"epoch_{}.pt".format(10),
              "output_dir_path":output_dir,
              "input_width_size":28,
              "input_height_size":28,
              "feature_size": 512,
              "num_labels": 10,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
1000 step processed.. current loss : 2.027773141860962
Average Loss : 2.07231116104126
Accuracy : 0.423
1000 step processed.. current loss : 1.7940324544906616
Average Loss : 1.8343861236572265
Accuracy : 0.5169
1000 step processed.. current loss : 1.7958078384399414
Average Loss : 1.759981601079305
Accuracy : 0.5304
1000 step processed.. current loss : 1.7023024559020996
Average Loss : 1.7212333463668823
Accuracy : 0.5157
1000 step processed.. current loss : 1.6659455299377441
Average Loss : 1.7019661098480225
Accuracy : 0.5508
1000 step processed.. current loss : 1.6877518892288208
Average Loss : 1.6918840938568116
Accuracy : 0.4802
1000 step processed.. current loss : 1.664292573928833
Average Loss : 1.6854314500172933
Accuracy : 0.4611
1000 step processed.. current loss : 1.6671111583709717
Average Loss : 1.679315783882141
Accuracy : 0.4576
1000 step processed.. current loss : 1.6657155752182007
Average Loss : 1.6672776

##### layer_10 : hidden layer 10개 사용

In [22]:
import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "trained_model_name":"epoch_{}.pt".format(10),
              "output_dir_path":output_dir,
              "input_width_size":28,
              "input_height_size":28,
              "feature_size": 512,
              "num_labels": 10,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
1000 step processed.. current loss : 2.292827606201172
Average Loss : 2.3037627063751223
Accuracy : 0.1135
1000 step processed.. current loss : 2.232616901397705
Average Loss : 2.2384571655273438
Accuracy : 0.2078
1000 step processed.. current loss : 2.0917508602142334
Average Loss : 2.100410018793742
Accuracy : 0.2106
1000 step processed.. current loss : 2.073847532272339
Average Loss : 2.0435284797668456
Accuracy : 0.2113
1000 step processed.. current loss : 1.9720739126205444
Average Loss : 2.0209314589182537
Accuracy : 0.211
1000 step processed.. current loss : 2.02812123298645
Average Loss : 2.0095047279993694
Accuracy : 0.209
1000 step processed.. current loss : 1.9392119646072388
Average Loss : 2.0005126370747885
Accuracy : 0.2106
1000 step processed.. current loss : 1.9526029825210571
Average Loss : 1.99466094900767
Accuracy : 0.2101
1000 step processed.. current loss : 1.9323465824127197
Average Loss : 1.9919044684

# 결과
|hidden_layer 갯수|loss|test_accuracy|
|------|---|---|
|layer_1|1.5550685330708822|0.8978|
|layer_3|1.5056964487075806|0.9549|
|layer_5|1.6560781340281168|0.4645|
|layer_10|1.989474365679423|0.2101|

accuracy가 가장 높을 때는 hidden layer를 3개 쌓을 때였다.  
무조건 깊게 쌓는다고 해서 성능이 올라가지는 않는다.